In [1]:
using InteractiveUtils, Test

In [2]:
function pw_rec(n, x)
    if iszero(n)
        one(x)
    else
        x * pw_rec(n - 1, x)
    end
end

pw_rec (generic function with 1 method)

In [3]:
pw_rec(3, "Abc")

"AbcAbcAbc"

In [4]:
Base.iszero(::Val{N}) where {N} = iszero(N)

In [5]:
Base.:-(::Val{n}, m) where {n} = Val{n - m}()

In [6]:
pw_rec(Val{3}(), 10)

1000

In [7]:
@code_typed pw_rec(Val{3}(), 10)

CodeInfo(
1 ─      nothing::Nothing
│   %2 = Base.mul_int(x, 1)::Int64
│   %3 = Base.mul_int(x, %2)::Int64
│   %4 = Base.mul_int(x, %3)::Int64
└──      return %4
) => Int64

In [8]:
abstract type MAbs{T} end

struct MOne{T} <: MAbs{T} end

struct MVal{T} <: MAbs{T}
    v::T
end

In [9]:
from_mabs(::MOne{T}) where {T} = one(T)
from_mabs(x::MVal{T}) where {T} = x.v

from_mabs (generic function with 2 methods)

In [10]:
from_mabs(MOne{Int}()), from_mabs(MVal("Abc"))

(1, "Abc")

In [11]:
to_mabs(x::T) where {T} =
    isone(x) ? MOne{T}() : MVal{T}(x)

to_mabs (generic function with 1 method)

In [12]:
to_mabs(""), to_mabs("Abc")

(MOne{String}(), MVal{String}("Abc"))

In [13]:
function pw_abs(n, x::MAbs{T}) where {T}
    if iszero(n)
        MOne{T}()
    else
        x * pw_abs(n - 1, x)
    end
end

pw_abs (generic function with 1 method)

In [14]:
Base.:*(::MOne{T}, y::MAbs{T}) where {T} = y
Base.:*(x::MVal{T}, y::MOne{T}) where {T} = x
Base.:*(x::MVal{T}, y::MVal{T}) where {T} = MVal{T}(x.v * y.v)

In [15]:
to_mabs("") * to_mabs(""), to_mabs("") * to_mabs("Abc"), to_mabs("Abc") * to_mabs("")

(MOne{String}(), MVal{String}("Abc"), MVal{String}("Abc"))

In [16]:
to_mabs("Abc") * to_mabs("Def")

MVal{String}("AbcDef")

In [17]:
pw_abs(Val(0), to_mabs("Abc"))

MOne{String}()

In [18]:
pw_abs(Val(1), to_mabs("Abc"))

MVal{String}("Abc")

In [19]:
pw_abs(Val(3), to_mabs("Abc"))

MVal{String}("AbcAbcAbc")

In [20]:
@code_typed pw_abs(Val(3), to_mabs(10))

CodeInfo(
1 ─      nothing::Nothing
│   %2 = Base.getfield(x, :v)::Int64
│   %3 = Base.getfield(x, :v)::Int64
│   %4 = Base.mul_int(%2, %3)::Int64
│   %5 = Base.getfield(x, :v)::Int64
│   %6 = Base.mul_int(%5, %4)::Int64
│   %7 = %new(MVal{Int64}, %6)::MVal{Int64}
└──      return %7
) => MVal{Int64}

In [23]:
abstract type GVal{T} end

struct GOne{T} <: GVal{T} end

struct GAny{T} <: GVal{T}
    e
end

In [24]:
from_gval(g::GOne{T}) where {T} = :($(one(T)))
from_gval(g::GAny{T}) where {T} = g.e

from_gval (generic function with 2 methods)

In [25]:
Base.:*(x::GOne{T}, y::GVal{T}) where {T} = y
Base.:*(x::GAny{T}, y::GOne{T}) where {T} = x
Base.:*(x::GAny{T}, y::GAny{T}) where {T} = GAny{T}(:($(x.e) * $(y.e)))

In [26]:
function pw_gen_impl(n, x::GAny{T}) where {T}
    if iszero(n)
        GOne{T}()
    else
        x * pw_gen_impl(n - 1, x)
    end
end

pw_gen_impl (generic function with 1 method)

In [27]:
Tuple(from_gval(pw_gen_impl(n, GAny{Int64}(:x))) for n in 0:3)

(1, :x, :(x * x), :(x * (x * x)))

In [28]:
@generated function pw_gen(::Val{n}, x::T) where {n, T}
    from_gval(pw_gen_impl(n, GAny{T}(:x)))
end

pw_gen (generic function with 1 method)

In [30]:
Tuple(pw_gen(Val(n), 10) for n in 0:3)

(1, 10, 100, 1000)